<h1> THE BATTLE OF NEIGHBOURHOODS </h1>

In [36]:
#Importing the required libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: | ^C
failed with initial frozen solve. Retrying with flexible solve.

CondaError: KeyboardInterrupt

failed

CondaError: KeyboardInterrupt

Libraries imported.


In [37]:
#Downloading the data

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [38]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [39]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 


# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

#extracting the features from the data
neighborhoods_data = newyork_data['features']

In [40]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [41]:
#getting the coordinates for New York
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [42]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [50]:
#foursquare credentials
CLIENT_ID = 'SJYYDFPICFOV0A2XLJ5M1PXCGMTH1PW32PT2KIRHL0ILQNKV' # your Foursquare ID
CLIENT_SECRET = '10X3SXO3QDNU24PE1CFOO4QZRWQAPNO2FQP1B3L15KDKS1YU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SJYYDFPICFOV0A2XLJ5M1PXCGMTH1PW32PT2KIRHL0ILQNKV
CLIENT_SECRET:10X3SXO3QDNU24PE1CFOO4QZRWQAPNO2FQP1B3L15KDKS1YU


In [71]:
address = 'New York,NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


In [72]:
search_query = 'restaurant'
radius = 500
print(search_query + ' .... OK!')

restaurant .... OK!


In [73]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=SJYYDFPICFOV0A2XLJ5M1PXCGMTH1PW32PT2KIRHL0ILQNKV&client_secret=10X3SXO3QDNU24PE1CFOO4QZRWQAPNO2FQP1B3L15KDKS1YU&ll=40.7127281,-74.0060152&v=20180605&query=restaurant&radius=500'

In [74]:
results = requests.get(url).json()

In [75]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,4b4fdfc8f964a520801827e3,TJ Byrnes Bar and Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591694716,False,77 Fulton St,Gold St,40.709233,-74.003747,"[{'label': 'display', 'lat': 40.70923312629616...",433,10038,US,New York,NY,United States,"[77 Fulton St (Gold St), New York, NY 10038, U...",131643631,NaN,NaN,NaN,NaN,NaN,NaN
1,45e5c256f964a52046431fe3,Mudville Restaurant & Tap House,"[{'id': '4bf58dd8d48988d14c941735', 'name': 'W...",v-1591694716,False,126 Chambers St,btwn W Broadway & Church St,40.715336,-74.008881,"[{'label': 'display', 'lat': 40.71533575723845...",377,10007,US,New York,NY,United States,[126 Chambers St (btwn W Broadway & Church St)...,43156651,299726,https://www.seamless.com/menu/mudville-restaur...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png
2,4bc238adf8219c744286b410,Amore's Pizza Restaurant,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1591694716,False,147 Chambers St,Hudson Street,40.715860,-74.009888,"[{'label': 'display', 'lat': 40.71585960614924...",477,10007,US,New York,NY,United States,"[147 Chambers St (Hudson Street), New York, NY...",NaN,1431324,https://www.seamless.com/menu/cafe-amores-pizz...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png
3,4c4890da1879c9b6cce6e143,New Shezan Restaurant,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",v-1591694716,False,183 Church St,btwn Duane & Reade St.,40.715789,-74.007227,"[{'label': 'display', 'lat': 40.71578949233164...",355,10007,US,New York,NY,United States,"[183 Church St (btwn Duane & Reade St.), New Y...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4b88199af964a520e6df31e3,Win Won Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1591694716,False,10 Liberty Pl,Maiden Lane,40.709193,-74.009344,"[{'label': 'display', 'lat': 40.709193, 'lng':...",483,10038,US,New York,NY,United States,"[10 Liberty Pl (Maiden Lane), New York, NY 100...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered.dropna()

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,TJ Byrnes Bar and Restaurant,Restaurant,77 Fulton St,Gold St,40.709233,-74.003747,"[{'label': 'display', 'lat': 40.70923312629616...",433,10038,US,New York,NY,United States,"[77 Fulton St (Gold St), New York, NY 10038, U...",4b4fdfc8f964a520801827e3
1,Mudville Restaurant & Tap House,Wings Joint,126 Chambers St,btwn W Broadway & Church St,40.715336,-74.008881,"[{'label': 'display', 'lat': 40.71533575723845...",377,10007,US,New York,NY,United States,[126 Chambers St (btwn W Broadway & Church St)...,45e5c256f964a52046431fe3
2,Amore's Pizza Restaurant,Pizza Place,147 Chambers St,Hudson Street,40.715860,-74.009888,"[{'label': 'display', 'lat': 40.71585960614924...",477,10007,US,New York,NY,United States,"[147 Chambers St (Hudson Street), New York, NY...",4bc238adf8219c744286b410
3,New Shezan Restaurant,Middle Eastern Restaurant,183 Church St,btwn Duane & Reade St.,40.715789,-74.007227,"[{'label': 'display', 'lat': 40.71578949233164...",355,10007,US,New York,NY,United States,"[183 Church St (btwn Duane & Reade St.), New Y...",4c4890da1879c9b6cce6e143
4,Win Won Restaurant,Chinese Restaurant,10 Liberty Pl,Maiden Lane,40.709193,-74.009344,"[{'label': 'display', 'lat': 40.709193, 'lng':...",483,10038,US,New York,NY,United States,"[10 Liberty Pl (Maiden Lane), New York, NY 100...",4b88199af964a520e6df31e3


In [81]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<img src="income_restaurants.png">